This notebook contains information about analysis on the given dataset for airline services.

Author : Umidjon Sattorov. Student at Mohirdev platform.

In [1]:
#Importing essential modules and packages
import pandas as pd 
import numpy as np 

import matplotlib.pyplot as plt 
import seaborn as sns

In [2]:
analysis_data = pd.read_csv(filepath_or_buffer = './data/analysis_ready_dataset.csv', sep = ',')
analysis_data.head(10)

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,Vistara,UK,Bangalore,Early_Morning,1,Night,Mumbai,Economy,14.25,21,7212
1,SpiceJet,SG,Hyderabad,Evening,0,Night,Kolkata,Economy,1.75,7,5292
2,Vistara,UK,Bangalore,Morning,1,Evening,Delhi,Business,9.58,5,60553
3,Vistara,UK,Kolkata,Morning,1,Evening,Hyderabad,Economy,6.75,28,5760
4,Indigo,6E,Chennai,Early_Morning,0,Morning,Mumbai,Economy,2.00,4,10712
5,Air_India,AI,Delhi,Afternoon,1,Morning,Chennai,Business,20.42,5,45257
6,Vistara,UK,Chennai,Evening,1,Morning,Mumbai,Economy,18.42,20,5054
7,Air_India,AI,Bangalore,Morning,0,Afternoon,Delhi,Business,2.58,30,32923
8,Vistara,UK,Bangalore,Early_Morning,1,Night,Hyderabad,Economy,15.92,12,11383
9,Air_India,AI,Chennai,Night,1,Afternoon,Mumbai,Economy,16.50,18,4357


In [3]:
#Average cost of flight for each class category
analysis_data.groupby('class')['price'].mean()

class
Business    52653.102289
Economy      6564.509852
Name: price, dtype: float64